INS
### Import libraries

In [1]:
import pandas as pd
import numpy as np
from playsound import playsound
import time
import gc as gc
start_time = time.time()

In [2]:
pd.set_option("max_colwidth", None)


In [3]:
path_df = "..\\data\\original\\"
path_processed = "..\\..\\..\\data_nps\\processed\\"
dashboard = "..\\data\\dashboard\\"
muestreo = "..\\..\\..\\data_nps\\muestreo\\"
sonido = "..\\figures\\"

In [4]:
!ls ../data/original/

nps_enero2019_febrero_2021.csv
nps_enero2019_febrero_2021.rar


In [5]:
df = pd.read_csv("../data/original/nps_enero2019_febrero_2021.csv", sep = ";", encoding="cp1252",low_memory=False)

In [6]:
len(df.CIF_ID.unique())

97491

In [7]:
#df[["CIF_ID", "DOCUMENT_TYPE","DOCUMENT_NUMBER"]].head()
#df[["CIF_ID", "DOCUMENT_TYPE","DOCUMENT_NUMBER"]].to_csv(muestreo+ "CIF_ID_polizas-enero19-diciembre2020.csv", index = True)

### TNPS_SCORE a Categorias

In [8]:
import re

## Columna Categoria

Categoria = Detractor | Neutro | Promotor

Es una columna que sale de una condición simple con respecto a la columna original del dataset TNPS_SCORE. 
¿Vale la pena hacerla automatizadamente o la levantamos con Tableau?
Prefiero automatimacamente pero se puede hacer con Tableau

In [9]:
nps_dict = {
    0:"Detractor",
    1:"Detractor",
    2:"Detractor",
    3:"Detractor",
    4:"Detractor",
    5:"Detractor",
    6:"Detractor",
    7:"Neutro",
    8:"Neutro",
    9:"Promotor",
    10:"Promotor"
}

In [10]:
df = df.fillna(0)

In [11]:
df["NPS_SEGMENT"] = df['TNPS_SCORE'].apply(lambda x: nps_dict[x])

### Date processing

In [12]:
f.CREATION_DATE = df["CREATION_DATE"].apply(lambda x: re.sub(r" .*","", x))
df.RESPONSE_DATE = df["RESPONSE_DATE"].apply(lambda x: re.sub(r" .*","", x))

In [13]:
df.CREATION_DATE = pd.to_datetime(df.CREATION_DATE, format = "%d/%m/%Y")

In [14]:
df.RESPONSE_DATE = pd.to_datetime(df.RESPONSE_DATE, format = "%d/%m/%Y")


In [15]:
#creo variable year_monty, i, 2019-01
df["year_month"] = df.RESPONSE_DATE.dt.to_period('M')

In [16]:
df.RESPONSE_DATE2 = df.RESPONSE_DATE

<ipython-input-16-beb75a92c2c2>:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.RESPONSE_DATE2 = df.RESPONSE_DATE


In [17]:
import re
import string

In [18]:
def basic_text_cleanup(text):
    """
    Convert to lowercase, remove punctuation, remove digits
    """
    #print(text)
    text = text.lower() #lower case
    text = re.sub("[^\w|\s]", " ",text) #replace non word and non space characters by a space
    text = re.sub("\s+", " ",text) #remove extra spaces
    text = re.sub("á", "a",text) #saco acentos
    text = re.sub("é", "e",text) #saco acentos
    text = re.sub("í", "i",text) #saco acentos
    text = re.sub("ó", "o",text) #saco acentos
    text = re.sub("ú", "u",text) #saco acentos
    return text

In [19]:
#reemplazo los valores nulos en comentarios por "", para que no me de error, la cunión basic_text_cleanup
df.loc[df.TNPS_COMMENT.isna(),"TNPS_COMMENT"] = ""

In [20]:
df["TNPS_COMMENT"] = df["TNPS_COMMENT"].astype(str)

In [21]:
df["comments_clean"] = df["TNPS_COMMENT"].apply(lambda x:basic_text_cleanup(x))

# Taggeo

## Diccionario

### Mapa Categorías y Atributos

In [22]:
txt = "me quiero dar de baja"

In [36]:
#testeos
test = pd.DataFrame({"comment":
                         ["X la resolución que tomaron con el siniestro que Tube en el mes de abril . Me chocaron de atrás en la Panamericana . Y la culpa fue mi",
                          "Testeo 2","rueda de auxilio otro choque",
                          "auxilio",
                          "No me va avisar", txt]})
test

,comment
0,X la resolución que tomaron con el siniestro que Tube en el mes de abril . Me chocaron de atrás en la Panamericana . Y la culpa fue mi
1,Testeo 2
2,rueda de auxilio otro choque
3,auxilio
4,No me va avisar
5,me quiero dar de baja


In [37]:

test.comment.str.contains(r"Pagaba x debito automatico y hace 3 meses no me cobraban y nadie me comento q estaba sin cobertura", #test Ok

                 regex =True)

0    False
1    False
2    False
3    False
4    False
5    False
Name: comment, dtype: bool

In [33]:
text = "la aca llego en el tiempo indicado acababa "

In [26]:
re.findall(r"aca+\s",text)

['aca ']

## Aclaracion

Los atributos y motivos no deberían ser positivos o negativos. Eso lo debería dar necesariamente el puntaje. Por lo tanto conformidad de un detractor significaría disconformidad.

In [114]:
keyDict ={
    "8mot_Atencion":r"atenc.*|atend.*|atiend.*|\b(?:respon.*\W+(?:\w+\W+){0,5}?nunca|nunca\W+(?:\w+\W+){0,5}?respon.*)\b|\b(?:respues.*\W+(?:\w+\W+){0,5}?nunca|nunca\W+(?:\w+\W+){0,5}?respues.*)\b|falta de interes|nadie me llamo|respue.*|emplea.*|amable|buen trato|mal trato|personal+\s", #test Ok
    "3mot_Document&gestion":r"^gesti.?\w|\Wgesti.?\w|\Wgesti.$|foto.*verific.*|verific.*(auto|coche|vehic)|verific.*|verifiq|documentacion|cupon|tarjeta plastica|factura|credencial|talon|resumen|domicilio|\b(?:comprobante\W+(?:\w+\W+){0,3}?seguro",
    "4mot_FacturacionPagos":r"cbu|factura|debit.*|visa+\s|bancaria|banco|\b(?:resumen\W+(?:\w+\W+){0,5}?tarjeta|tarjeta\W+(?:\w+\W+){0,5}?resumen)\b|bbva|santander|galicia|frances|pago|pagos|debiendo|debo|deuda|debito|\Wpago\W|^cuotas{0,1}|\Wcuotas{0,1}|cobraron|^cobro|cobra.*|comprobante", #test Ok
    "2mot_Siniestro":r"accidente|siniestro|choque|colision|destruccion|pericia\Waccid.?\w|\Waccid.$|\Wsiniest.?\w|\Wsiniest.$|choque|\Wrobo.?\w|\Wrobo.$|\Wchoc?\w|\Wchoc.$|\Wrob.?\w|\Wrob.$|\Whurt.?\w|\Whurt.$|denuncia|abogado|abogados|granizo|gomas",
    #test Ok
    "1mot_Grua":r"grua|auxicaja|auxi-caja|\Wauxilio\W|\Wauxilio$|aux.*mecan|remolque|remolc.*|acarreo|aca+\s|europ+\s|excedencia de km|excedio km|kilometro",
    #"mot_recupero":r"lo jack|lojack|rastreo|satelital",
    "7mot_Cobertura&poliza": r"cubr.*|cobert.*|recono.|\Wtaller.\w*|\Wtaller.*$|parabrisas{0,1}|cubiertas{0,1}|ruedas{0,1}|sin seguro|(dejaron|quede|no tuve)\W+(?:\w+\W+){0,5}?segu.*\b|siniestro|choque|rotura|destruccion|accidente|vidrio|rueda+\s|motor|poliza|cristales",
    #test Ok
    "5mot_Precio": r"precio|carocaro/w|carisimo|costo.*|\b(?:aumento.*\W+(?:\w+\W+){0,5}?cuota|cuota\W+(?:\w+\W+){0,5}?aumento.*)\b|bonific.*|rebaja|descuentos|\b(?:alta.*\W+(?:\w+\W+){0,5}?cuota|cuota\W+(?:\w+\W+){0,5}?alta.*)\b|precio|^precios{0,2}|\Wprecios{0,2}|^car(o|a|is)|\Wcar(o|a|is)|\Wcar(o|a|is)|costoso|\Wcost.?\w|\Wcost.$|^aument.?\w|\Waument.?\w|\Waument.$|cobraba|subieron la franquicia|barato|promociones",
    "6mot_Accesibilidad": r"comunic.*|comuniq.*|contact.*|\Wcontest.?\w|\Wcontest.$|respuesta|respondieron|responde|respondio|\b(?:respon.*\W+(?:\w+\W+){0,5}?nunca|nunca\W+(?:\w+\W+){0,5}?respon.*)\b|\b(?:respues.*\W+(?:\w+\W+){0,5}?nunca|nunca\W+(?:\w+\W+){0,5}?respues.*)\b|\b(?:nunca.*\W+(?:\w+\W+){0,5}?llama.*)|no he sido tenido en cuenta|(nunca|no)\W+(?:\w+\W+){0,5}?(llego|respon.*|respu.*|mensaj.*)\b",
    "4mot_Servicio":r"servicio|me asociarion|vehículo Toyota",
    #"mot_documentacion":r"documentacion|cupon|tarjeta plastica|factura|credencial|talon|resumen",
    "8atr_Demora":r"tard.*|demora|eternidad|esper.*|vengo haciendo hace|\b(?:hace.*\W+(?:\w+\W+){0,5}?mes.*|mes.*\W+(?:\w+\W+){0,5}?hace)\b|demora|lenta*?|lento*?|lentos|^lent(a|o)|\Wlent(a|o)|tardar|tardo|demora|eternidad|^esper.?\w|\Wesper.?\w|\Wesper.$|\d.(mes|hor|dia|sem|mes)|(estu.*|tuv.*)\W+(?:\w+\W+){0,5}?(hrs|hs|horas|.*hs)\b|demoro",
     #test Ok
    "1atr_Reiteracion":r"\d {0,3}vec|varias {0,3}vec",
    "6atr_Eficacia": r"^efic.?\w|\Wefic.?\w|\Wefic.$|^rapide.?\w|\Wrapide.?\w|\Wrapide.$|resolucion|solucion|rapido|agilidad|funciona|rapida|resolver|resolucion|siencillo|celeridad|funciona bien|resuel|resolvi|velocidad|agil",
    "4atr_Conformidad": r"conformidad|^mal(a|o)|\\Wmal(a|o)|malisi.*|No me tienen en cuenta como cliente|puta|mierda|sorete.*|forros|basura|porqueria|cagada|deastr.*|conforme|comforme|conforme|confiable|confianza|confiabilidad|muy ^buen(a|o){0,1}|\Wes buen(a|o){0,1}|excelen|perfecto|inconveniente|cumplen",
    "5atr_Sucursal": r"sucursa.*|locales.*",
    "2atr_Callcenter":r"atiendan|call center|call|contestador|escucha|grabaciones|internos|musi.*|ocupados|operador telefon.*|teléfono|\b(?:no\W+(?:\w+\W+){0,5}?contestan|contestan\W+(?:\w+\W+){0,5}?nadie)\b|grabacion.*|0800|telefono|telefonico|lineas|telefonistas",
    "3atr_Digital":r"^redes|whatsapp|mail|casilla|internet|whasapp|whatsap|clave|usuario|contrase|gmail|registrarme|ingresar|android|pagina|app|what.*|maquinas|gmail|subir fotos|wassap|aplicacion|visualizar|chat.*|acceder|plataforma|bot|wsapp|washap|wsp|web|on line|whas.*|was.*|WhatsApp",
    "1atr_Baja":r"falta de respeto|dar de baja|cambiarme de empresa|otras empresas|otra.* compa.*|cambiar de compania|zurich|den de baja|\b(?:me voy\W+(?:\w+\W+){0,5}?empresa|baja\W+(?:\w+\W+){0,3}?definitva)\b|\b(?:motivo\W+(?:\w+\W+){0,5}?baja|solicitar\W+(?:\w+\W+){0,3}?baja)\b|\b(?:baja\W+(?:\w+\W+){0,5}?total|cambiar\W+(?:\w+\W+){0,5}?empresa)\b|darle de baja|renovar|darle baja|engañ.*|estafa.*|dare baja|iunigo|evaluando cambiar.*|considerando baja.*|consderando cambiar.*|decepciono|deplorable.*|terrible.*|poca predisposicion|maltrato.*|horrible|doy de baja|di de baja", #test Ok
    #"atr_respuesta": r"respuesta|respondieron|responde|respondio|\b(?:respon.*\W+(?:\w+\W+){0,5}?nunca|nunca\W+(?:\w+\W+){0,5}?respon.*)\b|\b(?:respues.*\W+(?:\w+\W+){0,5}?nunca|nunca\W+(?:\w+\W+){0,5}?respues.*)\b|\b(?:nunca.*\W+(?:\w+\W+){0,5}?llama.*)|no he sido tenido en cuenta|(nunca|no)\W+(?:\w+\W+){0,5}?(llego|respon.*|respu.*|mensaj.*)\b", #test Ok
     #"atr_capacitación": r"capacitacion|conocimientos|falta de conocimiento|no sabia|no saben|ignorancia|ignorante|explicacion", #test Ok
     "atr_Informacion": r"info.*|explicacion|dudas|capacitacion|capacitado|ignorante" #test Ok
}

In [28]:
import collections
keyDict = collections.OrderedDict(sorted(keyDict.items()))

In [29]:
def tagKeyword(df, columnName, keyDict):
    """
    args:
        df: a pandas dataframe
        columnName: the column name where the keywords will be searched
        keyDict: a dictionary of keyword/ regex expresion. The function will use the keys to create new columns, and values
                for each key to evaluate regular expressions.
    returns: the input dataframe with new boolean columns that indicate of a searched keywords was found
    """
    for k, val in enumerate(keyDict):
        #print(val)
        #print(keyDict[val])
        df[val] = df[columnName].str.contains(keyDict[val], regex=True).view('i1') 
    return df

In [30]:
def tagKeyword(df, columnName, keyDict):
    """
    args:
        df: a pandas dataframe
        columnName: the column name where the keywords will be searched
        keyDict: a dictionary of keyword/ regex expresion. The function will use the keys to create new columns, and values
                for each key to evaluate regular expressions.
    returns: the input dataframe with new boolean columns that indicate of a searched keywords was found
    """
    for k, val in enumerate(keyDict):
        #print(val)
        #print(keyDict[val])
        df[val] = df[columnName].str.contains(keyDict[val], regex=True).view('i1') 
    return df

In [31]:
tagged = tagKeyword(df, "comments_clean", keyDict)

In [32]:
tagged.head(2)

,CREATION_DATE,RESPONSE_DATE,CURRENT_STATUS,ALL_LOG_NOTES_COMBINED,ALERT_STATUS_CHANGED_TIME,CASE_OWNER,FEEDBACK_CALL_DATE,FEEDBACK_CALL_TIME,FEEDBACK_CALL_OUTCOME,APPOINTMENT_FOR_NEXT_CALL,...,4mot_FacturacionPagos,4mot_Servicio,5atr_Sucursal,5mot_Precio,6atr_Eficacia,6mot_Accesibilidad,7mot_Cobertura&poliza,8atr_Demora,8mot_Atencion,atr_Informacion
0,2018-12-05,2019-01-02,Resolved,[2019-01-02 10:46:48] Alert Created: Detractor Consumer: AR; [2019-01-02 10:46:50] Alert status set to New; [2019-01-03 03:32:08] Case Edited: Dueño de este caso (por favor ingresar su nombre) · Fecha del Primer contacto · Hora del Primer contacto · Resultado del Primer contacto · Agenda de próximo contacto (araujo); [2019-01-03 03:59:07] Case Edited: Date of 2nd Feedback Call · Time of 2nd Feedback Call · Outcome of 2nd Feedback Call (araujo); [2019-01-03 04:17:49] Case Edited: Date of 3rd Feedback Call · Time of 3rd Feedback Call · Outcome of 3rd Feedback Call · Descripción detallada del contacto (araujo); [2019-01-03 08:05:51] Case Edited: Descripción detallada del contacto · Fuiste capaz de resolver el inconveniente? · El cliente se encontró satisfecho con el resultado? · Hay riesgos de que el cliente solicite la baja de su póliza? · El cliente desea ser contactado por un representante de Generali? · El cliente recibirá un regalo sorpresa? · Plan de acción para la resolución (araujo); [2019-01-03 08:05:57] Alert Closed (araujo); [2019-01-03 08:06:47] Note Added: 3 intentos de contacto fallidos. se envía mail para una próxima llamada. 5TA LLAMADA SIN EXITO. SE RE ENVIA MAIL DONDE SE LE SOLICITA PEDIR BAJA DEL SEGURO DE AP para dar curso al trámite (araujo); [2019-05-21 04:18:02] Alert Reopened (araujo); [2019-05-21 04:18:06] Alert Resolved (araujo); [2019-05-21 04:18:23] Alert Closed (araujo); [2019-05-21 06:43:03] Alert Reopened (araujo); [2019-05-21 06:43:36] Case Edited: Descripción detallada del contacto · Fuiste capaz de resolver el inconveniente? · El cliente se encontró satisfecho con el resultado? · Hay riesgos de que el cliente solicite la baja de su póliza? · El cliente desea ser contactado por un representante de Generali? · El cliente recibirá un regalo sorpresa? · Plan de acción para la resolución (araujo); [2019-05-21 06:43:42] Alert Resolved (araujo); [2019-05-21 06:43:54] Alert Closed (araujo); [2019-05-21 06:44:00] Alert Reopened (araujo); [2019-05-21 06:44:04] Alert Resolved (araujo),21/5/2019 06:44:04,vanni gorostiza,3/1/2019,1/12/2019 08:30:00,Not reachable,03/01/2019,...,0,0,0,0,0,0,0,0,0,0
1,2018-12-05,2019-01-03,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Creo columnas de "motivos" y "atributos"

In [33]:
#esta función es la que crea un variable con los motivos, y otra con los atributos. Hay que pasarselos de manera explícita
def tagTopic(df, newColName, columnNames, split_char ="|"):
    columnNames = np.array(columnNames)
    #print(columnNames)
    if split_char != "|":
        newNames = []
        for e in columnNames:
            name = e.split(split_char)[1]
            print(name)
            newNames.append(name)
        newNames = np.array(newNames)
    df[newColName] = "|"
    for i in range(df.shape[0]):
        #print("***********************************")
        #print(df.loc[i, columnNames])
        index = df.loc[i, columnNames]>0
        #print(newNames[index])
        df.at[i, newColName] = newNames[index]

In [34]:
tagged.columns

Index(['CREATION_DATE', 'RESPONSE_DATE', 'CURRENT_STATUS',
       'ALL_LOG_NOTES_COMBINED', 'ALERT_STATUS_CHANGED_TIME', 'CASE_OWNER',
       'FEEDBACK_CALL_DATE', 'FEEDBACK_CALL_TIME', 'FEEDBACK_CALL_OUTCOME',
       'APPOINTMENT_FOR_NEXT_CALL',
       ...
       '4mot_FacturacionPagos', '4mot_Servicio', '5atr_Sucursal',
       '5mot_Precio', '6atr_Eficacia', '6mot_Accesibilidad',
       '7mot_Cobertura&poliza', '8atr_Demora', '8mot_Atencion',
       'atr_Informacion'],
      dtype='object', length=197)

In [35]:
#guardo los nombres de las columnas con los motivos
motivos = tagged.columns[tagged.columns.str.contains("mot_")]

In [36]:
#guardo los nomrbes de las columnas con los atributos
atributos = tagged.columns[tagged.columns.str.contains("atr_")]

#### Tag de "motivos"

In [37]:
#creo la variable con los motivos
tagTopic(tagged, "motivos", motivos, "_")

Grua
Siniestro
Document&gestion
FacturacionPagos
Servicio
Precio
Accesibilidad
Cobertura&poliza
Atencion


In [38]:
tagged.tail(2)

,CREATION_DATE,RESPONSE_DATE,CURRENT_STATUS,ALL_LOG_NOTES_COMBINED,ALERT_STATUS_CHANGED_TIME,CASE_OWNER,FEEDBACK_CALL_DATE,FEEDBACK_CALL_TIME,FEEDBACK_CALL_OUTCOME,APPOINTMENT_FOR_NEXT_CALL,...,4mot_Servicio,5atr_Sucursal,5mot_Precio,6atr_Eficacia,6mot_Accesibilidad,7mot_Cobertura&poliza,8atr_Demora,8mot_Atencion,atr_Informacion,motivos
125603,2021-02-04,2021-02-04,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,[Servicio]
125604,2021-02-04,2021-02-04,Closed,[2021-02-04 15:49:53] Alert Created: Promoter Autoclose: AR; [2021-02-04 15:49:57] Alert Closed,4/2/2021 15:49:57,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,[Atencion]


#### Tag de "atributos"

In [39]:
tagTopic(tagged, "atributos", atributos, "_")

Baja
Reiteracion
Callcenter
Digital
Conformidad
Sucursal
Eficacia
Demora
Informacion


In [40]:
tagged.head(2)

,CREATION_DATE,RESPONSE_DATE,CURRENT_STATUS,ALL_LOG_NOTES_COMBINED,ALERT_STATUS_CHANGED_TIME,CASE_OWNER,FEEDBACK_CALL_DATE,FEEDBACK_CALL_TIME,FEEDBACK_CALL_OUTCOME,APPOINTMENT_FOR_NEXT_CALL,...,5atr_Sucursal,5mot_Precio,6atr_Eficacia,6mot_Accesibilidad,7mot_Cobertura&poliza,8atr_Demora,8mot_Atencion,atr_Informacion,motivos,atributos
0,2018-12-05,2019-01-02,Resolved,[2019-01-02 10:46:48] Alert Created: Detractor Consumer: AR; [2019-01-02 10:46:50] Alert status set to New; [2019-01-03 03:32:08] Case Edited: Dueño de este caso (por favor ingresar su nombre) · Fecha del Primer contacto · Hora del Primer contacto · Resultado del Primer contacto · Agenda de próximo contacto (araujo); [2019-01-03 03:59:07] Case Edited: Date of 2nd Feedback Call · Time of 2nd Feedback Call · Outcome of 2nd Feedback Call (araujo); [2019-01-03 04:17:49] Case Edited: Date of 3rd Feedback Call · Time of 3rd Feedback Call · Outcome of 3rd Feedback Call · Descripción detallada del contacto (araujo); [2019-01-03 08:05:51] Case Edited: Descripción detallada del contacto · Fuiste capaz de resolver el inconveniente? · El cliente se encontró satisfecho con el resultado? · Hay riesgos de que el cliente solicite la baja de su póliza? · El cliente desea ser contactado por un representante de Generali? · El cliente recibirá un regalo sorpresa? · Plan de acción para la resolución (araujo); [2019-01-03 08:05:57] Alert Closed (araujo); [2019-01-03 08:06:47] Note Added: 3 intentos de contacto fallidos. se envía mail para una próxima llamada. 5TA LLAMADA SIN EXITO. SE RE ENVIA MAIL DONDE SE LE SOLICITA PEDIR BAJA DEL SEGURO DE AP para dar curso al trámite (araujo); [2019-05-21 04:18:02] Alert Reopened (araujo); [2019-05-21 04:18:06] Alert Resolved (araujo); [2019-05-21 04:18:23] Alert Closed (araujo); [2019-05-21 06:43:03] Alert Reopened (araujo); [2019-05-21 06:43:36] Case Edited: Descripción detallada del contacto · Fuiste capaz de resolver el inconveniente? · El cliente se encontró satisfecho con el resultado? · Hay riesgos de que el cliente solicite la baja de su póliza? · El cliente desea ser contactado por un representante de Generali? · El cliente recibirá un regalo sorpresa? · Plan de acción para la resolución (araujo); [2019-05-21 06:43:42] Alert Resolved (araujo); [2019-05-21 06:43:54] Alert Closed (araujo); [2019-05-21 06:44:00] Alert Reopened (araujo); [2019-05-21 06:44:04] Alert Resolved (araujo),21/5/2019 06:44:04,vanni gorostiza,3/1/2019,1/12/2019 08:30:00,Not reachable,03/01/2019,...,0,0,0,0,0,0,0,0,[],[]
1,2018-12-05,2019-01-03,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,[],[]


In [41]:
#convierte las columnas recién creadas a strings porque eran numpy arrays
tagged["motivos"] = tagged["motivos"].astype(str)

In [42]:
tagged["motivos"].value_counts()

[]                                                                                                                                38532
['Atencion']                                                                                                                      32654
['Servicio']                                                                                                                       7885
['Accesibilidad' 'Atencion']                                                                                                       7469
['Accesibilidad']                                                                                                                  3491
                                                                                                                                  ...  
['Grua' 'Siniestro' 'Document&gestion' 'Servicio' 'Accesibilidad'\n 'Atencion']                                                       1
['Siniestro' 'Document&gestion' 'Servicio' 'Prec

In [43]:
tagged["atributos"] = tagged["atributos"].astype(str)

In [44]:
tagged.columns

Index(['CREATION_DATE', 'RESPONSE_DATE', 'CURRENT_STATUS',
       'ALL_LOG_NOTES_COMBINED', 'ALERT_STATUS_CHANGED_TIME', 'CASE_OWNER',
       'FEEDBACK_CALL_DATE', 'FEEDBACK_CALL_TIME', 'FEEDBACK_CALL_OUTCOME',
       'APPOINTMENT_FOR_NEXT_CALL',
       ...
       '5atr_Sucursal', '5mot_Precio', '6atr_Eficacia', '6mot_Accesibilidad',
       '7mot_Cobertura&poliza', '8atr_Demora', '8mot_Atencion',
       'atr_Informacion', 'motivos', 'atributos'],
      dtype='object', length=199)

In [45]:
tagged["atributos"].value_counts()

[]                                                                    57782
['Eficacia']                                                          31117
['Conformidad' 'Demora']                                               6509
['Conformidad']                                                        6087
['Demora']                                                             4622
                                                                      ...  
['Digital' 'Conformidad' 'Sucursal' 'Eficacia' 'Informacion']             1
['Reiteracion' 'Eficacia' 'Demora' 'Informacion']                         1
['Baja' 'Reiteracion' 'Digital' 'Conformidad' 'Demora']                   1
['Baja' 'Digital' 'Conformidad' 'Sucursal' 'Demora' 'Informacion']        1
['Baja' 'Callcenter' 'Digital' 'Eficacia' 'Demora' 'Informacion']         1
Name: atributos, Length: 288, dtype: int64

In [46]:
# chequeo as observaciones taggeadas por un motivo o atributo
tagged["detected"] =  ~np.logical_and(tagged["motivos"] == "[]", tagged["atributos"] == "[]")

In [47]:
tagged.groupby("NPS_SEGMENT")["detected"].mean()

NPS_SEGMENT
Detractor    0.872937
Neutro       0.808081
Promotor     0.866986
Name: detected, dtype: float64

### 1. Export dataset taggeado

tagged.to_csv(dashboard+ "1-tagged.csv")

### Dataframe de motivos

Este dataframe se va a visualizar en tableau/powerbi

In [48]:
topics = tagged.groupby(["year_month","motivos","NPS_SEGMENT"])["TNPS_SCORE"].count().unstack()

In [49]:
topics.replace(np.NaN, 0, inplace=True)

In [50]:
topics = topics.astype(int)

In [51]:
topics.head(2)

NPS_SEGMENT                                                 Detractor  Neutro  \
year_month motivos                                                              
2019-01    ['Accesibilidad' 'Atencion']                            25      45   
           ['Accesibilidad' 'Cobertura&poliza' 'Atencion']          2       0   

NPS_SEGMENT                                                 Promotor  
year_month motivos                                                    
2019-01    ['Accesibilidad' 'Atencion']                          189  
           ['Accesibilidad' 'Cobertura&poliza' 'Atencion']         3

In [52]:
topics.isna().sum()

NPS_SEGMENT
Detractor    0
Neutro       0
Promotor     0
dtype: int64

In [53]:
#creo otra matriz con el porcentaje de detractores, neutros, etc
topics_per = topics.div(topics.sum(axis=1), axis=0).round(2) *100

In [54]:
topics_per = topics_per.astype(int)

In [55]:
topics_per.head(2)

NPS_SEGMENT                                                 Detractor  Neutro  \
year_month motivos                                                              
2019-01    ['Accesibilidad' 'Atencion']                            10      17   
           ['Accesibilidad' 'Cobertura&poliza' 'Atencion']         40       0   

NPS_SEGMENT                                                 Promotor  
year_month motivos                                                    
2019-01    ['Accesibilidad' 'Atencion']                           73  
           ['Accesibilidad' 'Cobertura&poliza' 'Atencion']        60

In [56]:
topics_per.columns = "(%)"+ topics_per.columns

In [57]:
topics["total_surveys"] = topics.sum(axis=1)

In [58]:
#uno los dos data frames con val absolutos y porcentajes
topics = topics.merge(topics_per, left_index =True, right_index = True)

In [59]:
topics.reset_index(inplace=True)

In [60]:
topics.total_surveys = topics.total_surveys.astype(int)

In [61]:
topics["speech_tag"] = "motivos"

In [62]:
topics.rename(columns={"motivos": "topic"}, inplace=True)

In [63]:
topics.head(2)

NPS_SEGMENT,year_month,topic,Detractor,Neutro,Promotor,total_surveys,(%)Detractor,(%)Neutro,(%)Promotor,speech_tag
0,2019-01,['Accesibilidad' 'Atencion'],25,45,189,259,10,17,73,motivos
1,2019-01,['Accesibilidad' 'Cobertura&poliza' 'Atencion'],2,0,3,5,40,0,60,motivos


### 2. Export topicos


topics.to_csv(dashboard + "2-topics_monthly.csv")

### Montly view

In [64]:
#import seaborn as sns

In [65]:
monthly_nps = df.groupby(["year_month","NPS_SEGMENT"])["TNPS_SCORE"].count().unstack()

In [66]:
np.array(monthly_nps.sum(axis=1))

array([4201, 3766, 2934, 3116, 3594, 4723, 6305, 6072, 5996, 6053, 5109,
       5789, 6886, 5594, 3616, 4570, 6052, 6236, 5286, 5459, 5058, 5400,
       4163, 4071, 4356, 1200], dtype=int64)

In [67]:
monthly_nps_per = monthly_nps.div(monthly_nps.sum(axis=1), axis=0).round(2) *100

In [68]:
monthly_nps_per = monthly_nps_per.astype(int)

In [69]:
monthly_nps_per.columns = "(%)"+ monthly_nps_per.columns

In [70]:
monthly_nps = monthly_nps.merge(monthly_nps_per, left_index =True, right_index = True)

In [71]:
monthly_nps["total_surveys"] = monthly_nps.sum(axis=1)

In [72]:
monthly_nps.reset_index(inplace=True)

In [73]:
monthly_nps.head()

NPS_SEGMENT,year_month,Detractor,Neutro,Promotor,(%)Detractor,(%)Neutro,(%)Promotor,total_surveys
0,2019-01,773,910,2518,18,22,60,4301
1,2019-02,682,824,2260,18,22,60,3866
2,2019-03,532,609,1793,18,21,61,3034
3,2019-04,524,638,1954,17,20,63,3216
4,2019-05,603,748,2243,17,21,62,3694


### 3. Export cantidades mensuales

monthly_nps.to_csv(dashboard + "3-monthly_nps.csv", encoding = "utf-8")

## Motivos y atributos 

In [74]:
tagged.head(1)

,CREATION_DATE,RESPONSE_DATE,CURRENT_STATUS,ALL_LOG_NOTES_COMBINED,ALERT_STATUS_CHANGED_TIME,CASE_OWNER,FEEDBACK_CALL_DATE,FEEDBACK_CALL_TIME,FEEDBACK_CALL_OUTCOME,APPOINTMENT_FOR_NEXT_CALL,...,5mot_Precio,6atr_Eficacia,6mot_Accesibilidad,7mot_Cobertura&poliza,8atr_Demora,8mot_Atencion,atr_Informacion,motivos,atributos,detected
0,2018-12-05,2019-01-02,Resolved,[2019-01-02 10:46:48] Alert Created: Detractor Consumer: AR; [2019-01-02 10:46:50] Alert status set to New; [2019-01-03 03:32:08] Case Edited: Dueño de este caso (por favor ingresar su nombre) · Fecha del Primer contacto · Hora del Primer contacto · Resultado del Primer contacto · Agenda de próximo contacto (araujo); [2019-01-03 03:59:07] Case Edited: Date of 2nd Feedback Call · Time of 2nd Feedback Call · Outcome of 2nd Feedback Call (araujo); [2019-01-03 04:17:49] Case Edited: Date of 3rd Feedback Call · Time of 3rd Feedback Call · Outcome of 3rd Feedback Call · Descripción detallada del contacto (araujo); [2019-01-03 08:05:51] Case Edited: Descripción detallada del contacto · Fuiste capaz de resolver el inconveniente? · El cliente se encontró satisfecho con el resultado? · Hay riesgos de que el cliente solicite la baja de su póliza? · El cliente desea ser contactado por un representante de Generali? · El cliente recibirá un regalo sorpresa? · Plan de acción para la resolución (araujo); [2019-01-03 08:05:57] Alert Closed (araujo); [2019-01-03 08:06:47] Note Added: 3 intentos de contacto fallidos. se envía mail para una próxima llamada. 5TA LLAMADA SIN EXITO. SE RE ENVIA MAIL DONDE SE LE SOLICITA PEDIR BAJA DEL SEGURO DE AP para dar curso al trámite (araujo); [2019-05-21 04:18:02] Alert Reopened (araujo); [2019-05-21 04:18:06] Alert Resolved (araujo); [2019-05-21 04:18:23] Alert Closed (araujo); [2019-05-21 06:43:03] Alert Reopened (araujo); [2019-05-21 06:43:36] Case Edited: Descripción detallada del contacto · Fuiste capaz de resolver el inconveniente? · El cliente se encontró satisfecho con el resultado? · Hay riesgos de que el cliente solicite la baja de su póliza? · El cliente desea ser contactado por un representante de Generali? · El cliente recibirá un regalo sorpresa? · Plan de acción para la resolución (araujo); [2019-05-21 06:43:42] Alert Resolved (araujo); [2019-05-21 06:43:54] Alert Closed (araujo); [2019-05-21 06:44:00] Alert Reopened (araujo); [2019-05-21 06:44:04] Alert Resolved (araujo),21/5/2019 06:44:04,vanni gorostiza,3/1/2019,1/12/2019 08:30:00,Not reachable,03/01/2019,...,0,0,0,0,0,0,0,[],[],False


In [75]:
motivos = pd.DataFrame((tagged.motivos.str.split("'").tolist()), index= df.index)[[0,1,2,3,4,5,6]].rename(
    columns={
        0: "mot0",
        1: "mot1",
        2: "mot2",
        3: "mot3",
        4: "mot4",
        5: "mot5",
        6: "mot6"
    })

In [76]:
motivos

,mot0,mot1,mot2,mot3,mot4,mot5,mot6
0,[],None,None,None,None,None,None
1,[],None,None,None,None,None,None
2,[,Servicio,],None,None,None,None
3,[,Atencion,],None,None,None,None
4,[,Cobertura&poliza,],None,None,None,None
...,...,...,...,...,...,...,...
125600,[],None,None,None,None,None,None
125601,[],None,None,None,None,None,None
125602,[,Atencion,],None,None,None,None
125603,[,Servicio,],None,None,None,None


In [77]:
motivos = motivos[["mot1","mot3","mot5"]]

In [78]:
# Se agrega al dataset
df2 = pd.concat([tagged,motivos], axis=1)

## Atributos

In [79]:
atributos = pd.DataFrame((tagged.atributos.str.split("'").tolist()), index= df.index)[[0,1,2,3,4]].rename(
    columns={
        0: "atr0",
        1: "atr1",
        2: "atr2",
        3: "atr3",
        4: "atr4",
        5: "atr5",
        6: "atr6"
    })

In [80]:
atributos = atributos[["atr1","atr3"]]

In [81]:
df3 = pd.concat([df2,atributos], axis=1)

,motivos,atributos
0,[],[]
1,[],[]
2,['Servicio'],[]
3,['Atencion'],[]
4,['Cobertura&poliza'],['Demora']
...,...,...
125600,[],['Eficacia']
125601,[],['Eficacia']
125602,['Atencion'],['Eficacia']
125603,['Servicio'],[]


In [92]:
del df2
del tagged

NameError: name 'df2' is not defined

In [84]:

lst = [df3]
for column in lst:
    column.loc[column["AGE"] < 30,  'grupo_edad'] = 20
    column.loc[(column["AGE"] >= 30) & (column["AGE"] <= 39), 'grupo_edad'] = 30
    column.loc[(column["AGE"] >= 40) & (column["AGE"] <= 49), 'grupo_edad'] = 40
    column.loc[(column["AGE"] >= 50) & (column["AGE"] <= 59), 'grupo_edad'] = 50
    column.loc[(column["AGE"] >= 60) & (column["AGE"] <= 69), 'grupo_edad'] = 60
    column.loc[(column["AGE"] >= 70) & (column["AGE"] <= 79), 'grupo_edad'] = 70
    column.loc[column["AGE"] >= 80, 'grupo_edad'] = "80+"


In [85]:
lst = [df3]
for column in lst:
    column.loc[column["AGE"] < 20,  'rango_etario'] = 20
    column.loc[(column["AGE"] >= 20) & (column["AGE"] <= 25), 'rango_etario'] = "20-25"
    column.loc[(column["AGE"] >= 26) & (column["AGE"] <= 30), 'rango_etario'] = "26-30"
    column.loc[(column["AGE"] >= 31) & (column["AGE"] <= 35), 'rango_etario'] = "31-35"
    column.loc[(column["AGE"] >= 36) & (column["AGE"] <= 40), 'rango_etario'] = "36-40"
    column.loc[(column["AGE"] >= 41) & (column["AGE"] <= 45), 'rango_etario'] = "41-45"
    column.loc[(column["AGE"] >= 46) & (column["AGE"] <= 50), 'rango_etario'] = "46-50"
    column.loc[(column["AGE"] >= 51) & (column["AGE"] <= 55), 'rango_etario'] = "51-55"
    column.loc[(column["AGE"] >= 56) & (column["AGE"] <= 60), 'rango_etario'] = "56-60"
    column.loc[(column["AGE"] >= 61) & (column["AGE"] <= 65), 'rango_etario'] = "61-65"
    column.loc[(column["AGE"] >= 66) & (column["AGE"] <= 70), 'rango_etario'] = "66-70"
    column.loc[(column["AGE"] >= 71) & (column["AGE"] <= 75), 'rango_etario'] = "71-75"
    column.loc[(column["AGE"] >= 76) & (column["AGE"] <= 80), 'rango_etario'] = "76-80"
    column.loc[column["AGE"] >= 81, 'rango_etario'] = "80+"

In [88]:
df3.to_csv(dashboard+"nps_actualizado2020.csv")

In [93]:
df3[["motivos","atributos"]]

,motivos,atributos
0,[],[]
1,[],[]
2,['Servicio'],[]
3,['Atencion'],[]
4,['Cobertura&poliza'],['Demora']
...,...,...
125600,[],['Eficacia']
125601,[],['Eficacia']
125602,['Atencion'],['Eficacia']
125603,['Servicio'],[]


In [90]:
print("El script tarda", ((time.time() - start_time)/60), "minutos")
playsound(sonido+"war.mp3")
gc.collect()

El script tarda 10.512417471408844 minutos


615

In [ ]:
df.RESPONSE_DATE.max()

In [ ]:
df.TOUCHPOINT_NUMBER.unique()

In [ ]:
len(df3)

In [ ]:
not_detected = df3[df3.detected == False]

In [ ]:
na = len(not_detected)
total = len(df3)
porciento = na/total

print('La cantidad de comentarios no tageados son ', na)
print('Representa', porciento, " de la muestra")
print('El porcentaje de comentarios taggeados es ', 1-porciento, " de la muestra")

In [ ]:
not_detected.to_csv(dashboard + "sintaggeos2.csv")

In [ ]:
detractor = df3[df3["NPS_SEGMENT"] == "Detractor"]

## Lo importante es hacer esta Matriz mes a mes

In [ ]:
orange = sns.light_palette("red", as_cmap=True)
topicros= pd.crosstab(detractor['mot1'],detractor['atr1'])
topicros.style.background_gradient(cmap=orange)

In [ ]:
detractor.head(1)

In [ ]:
pd.crosstab(detractor['mot1'],detractor['mot3'])

In [ ]:
pd.crosstab(detractor['atr1'],detractor['atr3'])

In [ ]:
print("El script tarda", ((time.time() - start_time)/60), "minutos")
playsound(sonido+"war.mp3")
gc.collect()